# 🥸 Personas 101

## 🏁 Let's kick this off with the right packages

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha"
#r "nuget: YamlDotNet, 13.7.1"

## 🔥 Fire up the kernel

⚠️ Note that if you're going to use the function-calling capabilities of the kernel, you'll need a function-calling compatible model. Please refer to [this chart](https://platform.openai.com/docs/guides/function-calling) on OpenAI's site. Make sure your kernel is using a model that supports function calling.

* gpt-4
* gpt-4-1106-preview
* gpt-4-0613
* gpt-3.5-turbo
* gpt-3.5-turbo-1106
* gpt-3.5-turbo-0613

In [ ]:
// Load settings
#!import config/Settings.cs 
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey, orgId)
        .Build();
}

## 🧱 We lay the foundations for Personas to work. Names turn out to be useful.

In [ ]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

#pragma warning disable SKEXP0101

public class Personay
{
    public string Name { get; set; }
    public string Instructions { get; set; }
    public string Description { get; set; }
}

public class NameGenerator
{
    private List<string> names;
    private int currentIndex = -1;

    public NameGenerator()
    {
        // Initialize the list with ungendered names
        names = new List<string>
        {
            "Alex", "Jordan", "Taylor", "Morgan", "Casey",
            "Riley", "Jamie", "Avery", "Reese", "Skyler",
            "Quinn", "Peyton", "Cameron", "Sawyer", "Drew",
            "Charlie", "Emerson", "Dakota", "Parker", "Sidney"
        };
    }

    public string GetNextName()
    {
        // Increment the index and reset if it exceeds the list count
        currentIndex = (currentIndex + 1) % names.Count;
        return names[currentIndex];
    }
}

 // Track agents for clean-up
static readonly Dictionary<string, IAgent> s_agents = new();

IAgentThread? s_currentThread = null;

async Task<IAgent> CreateAgentAsync(string name, string instructions, string description)
{
    var agent = await new AgentBuilder()
                    .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                    .WithInstructions(instructions)
                    .WithName(name)
                    .WithDescription(description)
                    .BuildAsync();

    return Track(name, agent);
}

async Task CleanUpAsync()
{
    Console.WriteLine("🧽 Cleaning up ...");

    if (s_currentThread != null)
    {
        Console.WriteLine("Thread going away ...");
        s_currentThread.DeleteAsync();
        s_currentThread = null;
    }
    
    if (s_agents.Any())
    {
        Console.WriteLine("Agents going away ...");
        await Task.WhenAll(s_agents.Values.Select(agent => agent.DeleteAsync()));
        s_agents.Clear();
    }
}

IAgent Track(string name, IAgent agent)
{
    s_agents[name] = agent; // Add or update the agent in the dictionary
    return agent;
}


## 🥸 We grab a YAML persona with name, instructions, description

In [ ]:
string myTeam = "";

List<string> personasAvailable = ["ContentCreator", "DigitalMarketer"];

List<(string Name, string Instructions, string Description)> agentInfo = new();
NameGenerator nameGenerator = new NameGenerator();

foreach (var (a, i) in personasAvailable.Select((value, idx) => (value, idx + 1)))
{
    var yaml = File.ReadAllText($"Personas/{a}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance) // Use camel case naming convention
        .Build();
    var p = deserializer.Deserialize<Personay>(yaml);
    string fakeName = nameGenerator.GetNextName();
    string desc = $"{p.Name}: {p.Description}";
    string instr = p.Instructions;
    agentInfo.Add((fakeName, instr, desc));
    Console.WriteLine($"Agent defined: {fakeName}");
    myTeam += $"{i}) {fakeName} --> {desc}\n";
}

myTeam

## 🐣 We're ready to hatch the agents

In [ ]:
#pragma warning disable SKEXP0101

foreach (var (name, instructions, description) in agentInfo)
{
    await CreateAgentAsync(name, instructions, description);
}

s_agents

## 🧑‍🤝‍🧑 You can view the team in a more human-readable form

In [ ]:
myTeam

## 🔍 We can look at an individual agent (when there are many)

In [ ]:
s_agents["Alex"] // just replace 'Alex' with the name of the agent you want to use instead

## 🧵 Agents are run on threads. Let's make one.

In [ ]:
#pragma warning disable SKEXP0101

// note that threads aren't attached to specific agents
s_currentThread = await s_agents["Alex"].NewThreadAsync();

Console.WriteLine("Your thread 🧵 is ready!");

## 🏃 Your messages (as `user`) get sent to the thread, and get invoked by an agent

In [ ]:
#pragma warning disable SKEXP0101

string whichAgent = "Alex";

var messages = new string[]
{
//    "Write a 100 word blog post about the benefits of using a chatbot.",
//    "Write a headline for the blog post.",
//    "Write a binding contract in the state of Louisiana.",
    "Operate a forklift for me",
};

foreach (var response in messages.Select(m => s_currentThread.InvokeAsync(s_agents[whichAgent], m)))
{
    await foreach (var message in response)
    {
        // Console.WriteLine($"[{message.Id}]");
        string speaker = message.Role == "user" ? "You" : whichAgent;
        Console.WriteLine(Utils.WordWrap($"# {speaker}: {message.Content}",80));
    }   
}

## 💬 Messages from you can be added to the thread without invocation

We can, for instance, drop news stories into the thread like so:

In [ ]:
#pragma warning disable SKEXP0101

await s_currentThread.AddUserMessageAsync(
"""
- Interview subject: Ella Dupont, owner of Whiskers the cat.
- Location: French Quarter, New Orleans.
- Incident: Whiskers, a tabby cat, escaped from home early morning.
- Community Response: Local residents and shopkeepers reported sightings.
- Recognition: Whiskers identified from flyers distributed by Dupont.
- Recovery: Found near St. Louis Cathedral by tourists.
- Dupont's statement: Expresses gratitude towards the community for assistance in finding Whiskers. Highlights the close-knit nature of the neighborhood.
""");

await s_currentThread.AddUserMessageAsync(
"""
- Incident Type: Robbery
- Location: Downtown Commercial Bank, Main Street
- Time of Incident: Approximately 10:30 AM
- Suspects: Two individuals, descriptions vague; one reportedly wearing a dark hoodie and sunglasses, the other in a baseball cap.
- Method: Entered the bank, threatened the cashier with a note demanding money. No weapons seen.
- Response: Bank's alarm triggered, police arrived within minutes.
- Eyewitness Accounts: Bank customers describe suspects as calm, quick. No physical harm reported.
- Amount Stolen: Undisclosed sum of money, still being assessed by bank officials.
- Police Statement: Investigation underway, reviewing surveillance footage. Public urged to come forward with any information.
- Security Measures: Bank to review and enhance security protocols.
- Community Reaction: Local businesses express concern, increase vigilance.
""");

## 🏃 That way you can leverage what's in the thread in a subsequent invocation

And then direct `Alex` to write a story about anything that's been reported on the thread

In [ ]:
#pragma warning disable SKEXP0101

string whichAgent = "Alex";
string singleMessage = "Write a 100-word, short news article about a recently reported incident.";

var response = s_currentThread.InvokeAsync(s_agents[whichAgent], singleMessage);
await foreach (var message in response)
{
    string speaker = message.Role == "user" ? "You" : whichAgent;
    Console.WriteLine(Utils.WordWrap($"# {speaker}: {message.Content}", 80));
}

In [ ]:
#pragma warning disable SKEXP0101

string whichAgent = "Jordan";
string singleMessage = "What's the digital marketing strategy for this new blog post?";

var response = s_currentThread.InvokeAsync(s_agents[whichAgent], singleMessage);
await foreach (var message in response)
{
    string speaker = message.Role == "user" ? "You" : whichAgent;
    Console.WriteLine(Utils.WordWrap($"# {speaker}: {message.Content}", 80));
}

## 🥸 Agents make a lot of junk, so be sure to do a cleanup before you're done

In [ ]:
await CleanUpAsync();

# 💪 Let's go beyond turn-taking with agents

Here we implement the pattern where two agents are controlled by one coordinating agent.

## 📦 Gather the necessary personas

In [ ]:
string myTeam = "";

List<string> personasAvailable = ["ArtDirector","CopyWriter"];

List<(string Name, string Instructions, string Description)> agentInfo = new();
NameGenerator nameGenerator = new NameGenerator();

foreach (var (a, i) in personasAvailable.Select((value, idx) => (value, idx + 1)))
{
    var yaml = File.ReadAllText($"Personas/{a}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance) // Use camel case naming convention
        .Build();
    var p = deserializer.Deserialize<Personay>(yaml);
    string fakeName = nameGenerator.GetNextName();
    string desc = $"{p.Name}: {p.Description}";
    string instr = p.Instructions;
    agentInfo.Add((fakeName, instr, desc));
    Console.WriteLine($"Agent defined: {fakeName}");
    myTeam += $"{i}) {fakeName} --> {desc}\n";
}

myTeam

## 🐣 Hatch the agents

In [ ]:
#pragma warning disable SKEXP0101

foreach (var (name, instructions, description) in agentInfo)
{
    await CreateAgentAsync(name, instructions, description);
}

s_agents

## 💬 Let the coordinator get the agents to work together

In [ ]:
#pragma warning disable SKEXP0101

// Create coordinator agent to oversee collaboration
var coordinator =
    Track("Coordinator",
        await new AgentBuilder()
            .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
            .WithInstructions("Reply the provided concept and have the copy-writer generate an marketing idea (copy).  Then have the art-director reply to the copy-writer with a review of the copy.  Always include the source copy in any message.  Always include the art-director comments when interacting with the copy-writer.  Coordinate the repeated replies between the copy-writer and art-director until the art-director approves the copy.")
            .WithPlugin(s_agents["Jordan"].AsPlugin())
            .WithPlugin(s_agents["Alex"].AsPlugin())
            .BuildAsync());

// Invoke as a plugin function
var response = await coordinator.AsPlugin().InvokeAsync("concept: maps made out of egg cartons.");

// Display final result
Console.WriteLine(response);

## 🧼 It's always good to clean up

In [ ]:
await CleanUpAsync();

# 🥱 You're still here? 

If you come back in a bit, I'll be adding:

* Agents calling Plugins
* Plans running as Plugins to be called by the Agents